<a href="https://colab.research.google.com/github/gayun99/dacon_competition/blob/main/Catboost_(Final)_by%ED%98%9C%EB%AF%BC%EB%8B%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 구글 코랩 통일용

In [ ]:
import os
import sys
import platform
import random
import math

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
from typing import List ,Dict, Tuple

from datetime import datetime

In [ ]:
train_data = pd.read_csv("Jobcare_data/train.csv")
test_data = pd.read_csv("Jobcare_data/test.csv")

d_code = pd.read_csv('Jobcare_data/속성_D_코드.csv', index_col=0).T.to_dict()
h_code = pd.read_csv('Jobcare_data/속성_H_코드.csv', index_col=0).T.to_dict()
l_code = pd.read_csv('Jobcare_data/속성_L_코드.csv', index_col=0).T.to_dict()

print("train_data.shape: ", train_data.shape)
print("test_data.shape: ", test_data.shape)

train_data.shape:  (501951, 35)
test_data.shape:  (46404, 34)


In [ ]:
from typing import Dict
def add_code(
    df: pd.DataFrame,
    d_code: Dict[int, Dict[str, int]], 
    h_code: Dict[int, Dict[str, int]], 
    l_code: Dict[int, Dict[str, int]],
) -> pd.DataFrame:
    
    # Copy input data
    df = df.copy()   

    # D Code
    df['person_prefer_d_1_n'] = df['person_prefer_d_1'].apply(lambda x: d_code[x]['속성 D 세분류코드'])
    df['person_prefer_d_1_s'] = df['person_prefer_d_1'].apply(lambda x: d_code[x]['속성 D 소분류코드'])
    df['person_prefer_d_1_m'] = df['person_prefer_d_1'].apply(lambda x: d_code[x]['속성 D 중분류코드'])
    df['person_prefer_d_1_l'] = df['person_prefer_d_1'].apply(lambda x: d_code[x]['속성 D 대분류코드'])

    df['person_prefer_d_2_n'] = df['person_prefer_d_2'].apply(lambda x: d_code[x]['속성 D 세분류코드'])
    df['person_prefer_d_2_s'] = df['person_prefer_d_2'].apply(lambda x: d_code[x]['속성 D 소분류코드'])
    df['person_prefer_d_2_m'] = df['person_prefer_d_2'].apply(lambda x: d_code[x]['속성 D 중분류코드'])
    df['person_prefer_d_2_l'] = df['person_prefer_d_2'].apply(lambda x: d_code[x]['속성 D 대분류코드'])

    df['person_prefer_d_3_n'] = df['person_prefer_d_3'].apply(lambda x: d_code[x]['속성 D 세분류코드'])
    df['person_prefer_d_3_s'] = df['person_prefer_d_3'].apply(lambda x: d_code[x]['속성 D 소분류코드'])
    df['person_prefer_d_3_m'] = df['person_prefer_d_3'].apply(lambda x: d_code[x]['속성 D 중분류코드'])
    df['person_prefer_d_3_l'] = df['person_prefer_d_3'].apply(lambda x: d_code[x]['속성 D 대분류코드'])

    df['contents_attribute_d_n'] = df['contents_attribute_d'].apply(lambda x: d_code[x]['속성 D 세분류코드'])
    df['contents_attribute_d_s'] = df['contents_attribute_d'].apply(lambda x: d_code[x]['속성 D 소분류코드'])
    df['contents_attribute_d_m'] = df['contents_attribute_d'].apply(lambda x: d_code[x]['속성 D 중분류코드'])
    df['contents_attribute_d_l'] = df['contents_attribute_d'].apply(lambda x: d_code[x]['속성 D 대분류코드'])

    # H Code
    df['person_prefer_h_1_l'] = df['person_prefer_h_1'].apply(lambda x: h_code[x]['속성 H 대분류코드'])
    df['person_prefer_h_1_m'] = df['person_prefer_h_1'].apply(lambda x: h_code[x]['속성 H 중분류코드'])
    
    df['person_prefer_h_2_l'] = df['person_prefer_h_2'].apply(lambda x: h_code[x]['속성 H 대분류코드'])
    df['person_prefer_h_2_m'] = df['person_prefer_h_2'].apply(lambda x: h_code[x]['속성 H 중분류코드'])
    
    df['person_prefer_h_3_l'] = df['person_prefer_h_3'].apply(lambda x: h_code[x]['속성 H 대분류코드'])
    df['person_prefer_h_3_m'] = df['person_prefer_h_3'].apply(lambda x: h_code[x]['속성 H 중분류코드'])

    df['contents_attribute_h_l'] = df['contents_attribute_h'].apply(lambda x: h_code[x]['속성 H 대분류코드'])
    df['contents_attribute_h_m'] = df['contents_attribute_h'].apply(lambda x: h_code[x]['속성 H 중분류코드'])

    # L Code
    df['contents_attribute_l_n'] = df['contents_attribute_l'].apply(lambda x: l_code[x]['속성 L 세분류코드'])
    df['contents_attribute_l_s'] = df['contents_attribute_l'].apply(lambda x: l_code[x]['속성 L 소분류코드'])
    df['contents_attribute_l_m'] = df['contents_attribute_l'].apply(lambda x: l_code[x]['속성 L 중분류코드'])
    df['contents_attribute_l_l'] = df['contents_attribute_l'].apply(lambda x: l_code[x]['속성 L 대분류코드'])
    
    return df

train_data = add_code(train_data, d_code, h_code, l_code)
test_data = add_code(test_data, d_code, h_code, l_code)
print("train_data.shape: ", train_data.shape)
print("test_data.shape: ", test_data.shape)

train_data.shape:  (501951, 63)
test_data.shape:  (46404, 62)


In [ ]:
job_data = train_data.copy()
job_data_test = test_data.copy()

In [ ]:
X = job_data.drop(['id','person_prefer_f', 'person_prefer_g', 'contents_open_dt', 'person_rn', 'contents_rn','target'],axis = 1)
target = job_data['target']

job_data_test = job_data_test.drop(['id','person_prefer_f', 'person_prefer_g', 'contents_open_dt', 'person_rn', 'contents_rn'],axis = 1)

In [ ]:
cat_features = X.columns[X.nunique() > 2].tolist()

In [ ]:
# 학습데이터 분리
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,target,
                                                 test_size=0.2,random_state=42, stratify = target)

---

## 랜덤한 파라미터 대입한 모델링

In [ ]:
import catboost as cb
from catboost import cv, Pool, CatBoostClassifier
cb_model = CatBoostClassifier(task_type='CPU', iterations = 150,
                              random_state = 42, 
                              eval_metric="F1")

In [ ]:
cb_model.fit(X_train, y_train, cat_features= cat_features, plot = True,
            eval_set=(X_test, y_test))

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

Learning rate set to 0.318006
0:	learn: 0.6250642	test: 0.6272690	best: 0.6272690 (0)	total: 221ms	remaining: 32.9s
1:	learn: 0.6075590	test: 0.6048413	best: 0.6272690 (0)	total: 518ms	remaining: 38.3s
2:	learn: 0.6129553	test: 0.6096984	best: 0.6272690 (0)	total: 764ms	remaining: 37.5s
3:	learn: 0.6145074	test: 0.6107206	best: 0.6272690 (0)	total: 965ms	remaining: 35.2s
4:	learn: 0.6142385	test: 0.6100451	best: 0.6272690 (0)	total: 1.16s	remaining: 33.7s
5:	learn: 0.6158785	test: 0.6120379	best: 0.6272690 (0)	total: 1.36s	remaining: 32.7s
6:	learn: 0.6166800	test: 0.6127067	best: 0.6272690 (0)	total: 1.54s	remaining: 31.5s
7:	learn: 0.6195864	test: 0.6163382	best: 0.6272690 (0)	total: 1.77s	remaining: 31.4s
8:	learn: 0.6200300	test: 0.6165914	best: 0.6272690 (0)	total: 1.95s	remaining: 30.6s
9:	learn: 0.6207309	test: 0.6159125	best: 0.6272690 (0)	total: 2.12s	remaining: 29.6s
10:	learn: 0.6212590	test: 0.6168938	best: 0.6272690 (0)	total: 2.32s	remaining: 29.4s
11:	learn: 0.6217165	te

95:	learn: 0.6396669	test: 0.6331800	best: 0.6335417 (87)	total: 18.6s	remaining: 10.5s
96:	learn: 0.6397763	test: 0.6331457	best: 0.6335417 (87)	total: 18.8s	remaining: 10.3s
97:	learn: 0.6397842	test: 0.6329898	best: 0.6335417 (87)	total: 19s	remaining: 10.1s
98:	learn: 0.6401489	test: 0.6331677	best: 0.6335417 (87)	total: 19.2s	remaining: 9.88s
99:	learn: 0.6402270	test: 0.6330392	best: 0.6335417 (87)	total: 19.4s	remaining: 9.7s
100:	learn: 0.6402068	test: 0.6330469	best: 0.6335417 (87)	total: 19.6s	remaining: 9.53s
101:	learn: 0.6407074	test: 0.6334254	best: 0.6335417 (87)	total: 19.8s	remaining: 9.33s
102:	learn: 0.6408144	test: 0.6334473	best: 0.6335417 (87)	total: 20.1s	remaining: 9.15s
103:	learn: 0.6409257	test: 0.6333840	best: 0.6335417 (87)	total: 20.3s	remaining: 8.99s
104:	learn: 0.6410365	test: 0.6331378	best: 0.6335417 (87)	total: 20.6s	remaining: 8.81s
105:	learn: 0.6412927	test: 0.6334906	best: 0.6335417 (87)	total: 20.8s	remaining: 8.62s
106:	learn: 0.6412914	test: 0

## 파라미터 찾기 위해 GridSearch

In [ ]:
from sklearn.metrics import make_scorer, accuracy_score
from sklearn.model_selection import GridSearchCV

In [ ]:
clf = CatBoostClassifier()
params = {'iterations': [100,150],
          'learning_rate' : [0.03,0.05],
          'max_depth': [8, 9, 10],        #보통 4~10
          'random_seed': [42]
         }
scorer = make_scorer(accuracy_score)
clf_grid = GridSearchCV(estimator=clf, param_grid=params, scoring=scorer, cv=5)

In [ ]:
clf_grid.fit(X_train, y_train)
best_param = clf_grid.best_params_
best_param

0:	learn: 0.6916936	total: 33.4ms	remaining: 1.3s
1:	learn: 0.6903711	total: 62.1ms	remaining: 1.18s
2:	learn: 0.6890866	total: 91.6ms	remaining: 1.13s
3:	learn: 0.6878470	total: 121ms	remaining: 1.09s
4:	learn: 0.6866896	total: 150ms	remaining: 1.05s
5:	learn: 0.6856681	total: 179ms	remaining: 1.01s
6:	learn: 0.6846109	total: 207ms	remaining: 975ms
7:	learn: 0.6837046	total: 235ms	remaining: 940ms
8:	learn: 0.6828432	total: 266ms	remaining: 916ms
9:	learn: 0.6819801	total: 296ms	remaining: 887ms
10:	learn: 0.6811680	total: 329ms	remaining: 868ms
11:	learn: 0.6804436	total: 359ms	remaining: 839ms
12:	learn: 0.6797413	total: 388ms	remaining: 806ms
13:	learn: 0.6790612	total: 418ms	remaining: 776ms
14:	learn: 0.6783900	total: 448ms	remaining: 747ms
15:	learn: 0.6777668	total: 476ms	remaining: 714ms
16:	learn: 0.6771621	total: 509ms	remaining: 689ms
17:	learn: 0.6766011	total: 537ms	remaining: 656ms
18:	learn: 0.6760557	total: 567ms	remaining: 627ms
19:	learn: 0.6755164	total: 598ms	remai

7:	learn: 0.6837210	total: 234ms	remaining: 938ms
8:	learn: 0.6828295	total: 265ms	remaining: 914ms
9:	learn: 0.6819880	total: 295ms	remaining: 884ms
10:	learn: 0.6811665	total: 324ms	remaining: 855ms
11:	learn: 0.6803784	total: 351ms	remaining: 820ms
12:	learn: 0.6796422	total: 381ms	remaining: 791ms
13:	learn: 0.6789413	total: 409ms	remaining: 760ms
14:	learn: 0.6782559	total: 438ms	remaining: 730ms
15:	learn: 0.6776448	total: 468ms	remaining: 702ms
16:	learn: 0.6770326	total: 495ms	remaining: 670ms
17:	learn: 0.6764631	total: 524ms	remaining: 641ms
18:	learn: 0.6758942	total: 553ms	remaining: 611ms
19:	learn: 0.6753566	total: 580ms	remaining: 580ms
20:	learn: 0.6748907	total: 610ms	remaining: 552ms
21:	learn: 0.6744090	total: 638ms	remaining: 522ms
22:	learn: 0.6739596	total: 667ms	remaining: 493ms
23:	learn: 0.6735096	total: 694ms	remaining: 463ms
24:	learn: 0.6731170	total: 724ms	remaining: 435ms
25:	learn: 0.6726924	total: 753ms	remaining: 406ms
26:	learn: 0.6723063	total: 782ms	

12:	learn: 0.6791682	total: 499ms	remaining: 1.04s
13:	learn: 0.6784541	total: 536ms	remaining: 996ms
14:	learn: 0.6777439	total: 573ms	remaining: 955ms
15:	learn: 0.6770776	total: 609ms	remaining: 914ms
16:	learn: 0.6763753	total: 647ms	remaining: 876ms
17:	learn: 0.6757842	total: 685ms	remaining: 837ms
18:	learn: 0.6751594	total: 722ms	remaining: 798ms
19:	learn: 0.6746297	total: 762ms	remaining: 762ms
20:	learn: 0.6740559	total: 798ms	remaining: 722ms
21:	learn: 0.6735763	total: 836ms	remaining: 684ms
22:	learn: 0.6730476	total: 873ms	remaining: 645ms
23:	learn: 0.6725708	total: 908ms	remaining: 605ms
24:	learn: 0.6721812	total: 943ms	remaining: 566ms
25:	learn: 0.6717291	total: 979ms	remaining: 527ms
26:	learn: 0.6713376	total: 1.01s	remaining: 489ms
27:	learn: 0.6709363	total: 1.05s	remaining: 451ms
28:	learn: 0.6705480	total: 1.09s	remaining: 412ms
29:	learn: 0.6701838	total: 1.12s	remaining: 375ms
30:	learn: 0.6698305	total: 1.16s	remaining: 337ms
31:	learn: 0.6694428	total: 1.2

15:	learn: 0.6759992	total: 1.14s	remaining: 1.71s
16:	learn: 0.6753719	total: 1.22s	remaining: 1.64s
17:	learn: 0.6747423	total: 1.29s	remaining: 1.57s
18:	learn: 0.6740755	total: 1.36s	remaining: 1.5s
19:	learn: 0.6734687	total: 1.44s	remaining: 1.44s
20:	learn: 0.6729196	total: 1.51s	remaining: 1.37s
21:	learn: 0.6723221	total: 1.59s	remaining: 1.3s
22:	learn: 0.6717942	total: 1.67s	remaining: 1.24s
23:	learn: 0.6712961	total: 1.75s	remaining: 1.17s
24:	learn: 0.6708416	total: 1.83s	remaining: 1.1s
25:	learn: 0.6703489	total: 1.92s	remaining: 1.03s
26:	learn: 0.6699055	total: 1.99s	remaining: 959ms
27:	learn: 0.6694610	total: 2.06s	remaining: 883ms
28:	learn: 0.6690310	total: 2.13s	remaining: 809ms
29:	learn: 0.6686129	total: 2.2s	remaining: 734ms
30:	learn: 0.6681996	total: 2.27s	remaining: 660ms
31:	learn: 0.6677882	total: 2.35s	remaining: 587ms
32:	learn: 0.6674152	total: 2.42s	remaining: 513ms
33:	learn: 0.6670456	total: 2.49s	remaining: 439ms
34:	learn: 0.6666310	total: 2.56s	r

21:	learn: 0.6689251	total: 652ms	remaining: 534ms
22:	learn: 0.6685088	total: 683ms	remaining: 505ms
23:	learn: 0.6680632	total: 714ms	remaining: 476ms
24:	learn: 0.6677011	total: 743ms	remaining: 446ms
25:	learn: 0.6672507	total: 775ms	remaining: 417ms
26:	learn: 0.6668683	total: 806ms	remaining: 388ms
27:	learn: 0.6664522	total: 836ms	remaining: 358ms
28:	learn: 0.6661113	total: 865ms	remaining: 328ms
29:	learn: 0.6657477	total: 894ms	remaining: 298ms
30:	learn: 0.6654039	total: 927ms	remaining: 269ms
31:	learn: 0.6650973	total: 956ms	remaining: 239ms
32:	learn: 0.6647930	total: 984ms	remaining: 209ms
33:	learn: 0.6644736	total: 1.02s	remaining: 179ms
34:	learn: 0.6641708	total: 1.04s	remaining: 149ms
35:	learn: 0.6638816	total: 1.07s	remaining: 119ms
36:	learn: 0.6635914	total: 1.1s	remaining: 89.6ms
37:	learn: 0.6633642	total: 1.14s	remaining: 59.8ms
38:	learn: 0.6631128	total: 1.17s	remaining: 29.9ms
39:	learn: 0.6628981	total: 1.19s	remaining: 0us
0:	learn: 0.6907620	total: 33.7

...

42:	learn: 0.6626031	total: 1.36s	remaining: 222ms
43:	learn: 0.6623330	total: 1.4s	remaining: 191ms
44:	learn: 0.6621272	total: 1.43s	remaining: 159ms
45:	learn: 0.6619259	total: 1.46s	remaining: 127ms
46:	learn: 0.6616904	total: 1.49s	remaining: 95.4ms
47:	learn: 0.6615097	total: 1.53s	remaining: 63.6ms
48:	learn: 0.6613711	total: 1.55s	remaining: 31.7ms
49:	learn: 0.6611607	total: 1.58s	remaining: 0us
0:	learn: 0.6907530	total: 32.2ms	remaining: 1.57s
1:	learn: 0.6886113	total: 62.9ms	remaining: 1.51s
2:	learn: 0.6867133	total: 98.7ms	remaining: 1.55s
3:	learn: 0.6849705	total: 135ms	remaining: 1.55s
4:	learn: 0.6833503	total: 169ms	remaining: 1.52s
5:	learn: 0.6818890	total: 203ms	remaining: 1.49s
6:	learn: 0.6805558	total: 234ms	remaining: 1.44s
7:	learn: 0.6793375	total: 270ms	remaining: 1.42s
8:	learn: 0.6782437	total: 308ms	remaining: 1.4s
9:	learn: 0.6772535	total: 347ms	remaining: 1.39s
10:	learn: 0.6763466	total: 381ms	remaining: 1.35s
11:	learn: 0.6754689	total: 413ms	remai

6:	learn: 0.6800403	total: 290ms	remaining: 1.78s
7:	learn: 0.6787274	total: 334ms	remaining: 1.75s
8:	learn: 0.6775541	total: 374ms	remaining: 1.71s
9:	learn: 0.6764331	total: 414ms	remaining: 1.65s
10:	learn: 0.6754065	total: 454ms	remaining: 1.61s
11:	learn: 0.6744920	total: 495ms	remaining: 1.57s
12:	learn: 0.6736228	total: 540ms	remaining: 1.54s
13:	learn: 0.6728200	total: 581ms	remaining: 1.49s
14:	learn: 0.6720427	total: 619ms	remaining: 1.44s
15:	learn: 0.6713575	total: 661ms	remaining: 1.4s
16:	learn: 0.6707211	total: 702ms	remaining: 1.36s
17:	learn: 0.6700259	total: 743ms	remaining: 1.32s
18:	learn: 0.6694055	total: 787ms	remaining: 1.28s
19:	learn: 0.6688469	total: 827ms	remaining: 1.24s
20:	learn: 0.6682368	total: 867ms	remaining: 1.2s
21:	learn: 0.6676849	total: 907ms	remaining: 1.15s
22:	learn: 0.6671345	total: 950ms	remaining: 1.11s
23:	learn: 0.6666896	total: 991ms	remaining: 1.07s
24:	learn: 0.6662792	total: 1.03s	remaining: 1.03s
25:	learn: 0.6658467	total: 1.07s	rem

18:	learn: 0.6678846	total: 1.49s	remaining: 2.44s
19:	learn: 0.6671573	total: 1.58s	remaining: 2.37s
20:	learn: 0.6665201	total: 1.66s	remaining: 2.29s
21:	learn: 0.6658162	total: 1.74s	remaining: 2.21s
22:	learn: 0.6652488	total: 1.81s	remaining: 2.13s
23:	learn: 0.6646738	total: 1.89s	remaining: 2.05s
24:	learn: 0.6641474	total: 1.97s	remaining: 1.97s
25:	learn: 0.6635625	total: 2.05s	remaining: 1.9s
26:	learn: 0.6631331	total: 2.13s	remaining: 1.82s
27:	learn: 0.6627799	total: 2.21s	remaining: 1.74s
28:	learn: 0.6623265	total: 2.29s	remaining: 1.66s
29:	learn: 0.6618960	total: 2.37s	remaining: 1.58s
30:	learn: 0.6614937	total: 2.45s	remaining: 1.5s
31:	learn: 0.6610628	total: 2.53s	remaining: 1.42s
32:	learn: 0.6606742	total: 2.61s	remaining: 1.34s
33:	learn: 0.6603299	total: 2.69s	remaining: 1.27s
34:	learn: 0.6599988	total: 2.77s	remaining: 1.19s
35:	learn: 0.6596291	total: 2.85s	remaining: 1.11s
36:	learn: 0.6592548	total: 2.94s	remaining: 1.03s
37:	learn: 0.6588970	total: 3.02s

32:	learn: 0.6607796	total: 2.79s	remaining: 1.44s
33:	learn: 0.6604080	total: 2.87s	remaining: 1.35s
34:	learn: 0.6599771	total: 2.95s	remaining: 1.26s
35:	learn: 0.6595880	total: 3.04s	remaining: 1.18s
36:	learn: 0.6592861	total: 3.13s	remaining: 1.1s
37:	learn: 0.6589205	total: 3.22s	remaining: 1.02s
38:	learn: 0.6586173	total: 3.31s	remaining: 933ms
39:	learn: 0.6583007	total: 3.39s	remaining: 848ms
40:	learn: 0.6580239	total: 3.47s	remaining: 763ms
41:	learn: 0.6576916	total: 3.55s	remaining: 677ms
42:	learn: 0.6574085	total: 3.64s	remaining: 592ms
43:	learn: 0.6570830	total: 3.72s	remaining: 507ms
44:	learn: 0.6567725	total: 3.81s	remaining: 423ms
45:	learn: 0.6565580	total: 3.89s	remaining: 338ms
46:	learn: 0.6563360	total: 3.98s	remaining: 254ms
47:	learn: 0.6560353	total: 4.09s	remaining: 170ms
48:	learn: 0.6557167	total: 4.2s	remaining: 85.7ms
49:	learn: 0.6554953	total: 4.29s	remaining: 0us
0:	learn: 0.6905048	total: 82.9ms	remaining: 4.06s
1:	learn: 0.6883149	total: 159ms	r

{'iterations': 50, 'learning_rate': 0.05, 'max_depth': 10, 'random_seed': 42}

## 찾은 파라미터 대입해서 다시 모델링

In [ ]:
cb_model_final = CatBoostClassifier(iterations = 250,  #이터레이션은 고고익선이라 일단 높여봄..별도설명없음..
                                    learning_rate = 0.05,
                                    loss_function = 'CrossEntropy', #binary 타겟이므로
                                    max_depth = 10,
                                    random_state = 42,
                                   eval_metric="F1")

cb_model_final.fit(X_train, y_train, cat_features= cat_features, plot = True, eval_set=(X_test, y_test))

y_pred_final = cb_model_final.predict(job_data_test)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 0.6174763	test: 0.6123371	best: 0.6123371 (0)	total: 1.8s	remaining: 7m 28s
1:	learn: 0.6336529	test: 0.6319594	best: 0.6319594 (1)	total: 3.79s	remaining: 7m 50s
2:	learn: 0.6344745	test: 0.6319516	best: 0.6319594 (1)	total: 5.63s	remaining: 7m 43s
3:	learn: 0.6359083	test: 0.6338658	best: 0.6338658 (3)	total: 7.15s	remaining: 7m 19s
4:	learn: 0.6350183	test: 0.6337314	best: 0.6338658 (3)	total: 8.75s	remaining: 7m 8s
5:	learn: 0.6334633	test: 0.6316282	best: 0.6338658 (3)	total: 10.5s	remaining: 7m 8s
6:	learn: 0.6344429	test: 0.6327340	best: 0.6338658 (3)	total: 12.3s	remaining: 7m 8s
7:	learn: 0.6353566	test: 0.6330499	best: 0.6338658 (3)	total: 14s	remaining: 7m 3s
8:	learn: 0.6352241	test: 0.6329534	best: 0.6338658 (3)	total: 15.7s	remaining: 6m 59s
9:	learn: 0.6358182	test: 0.6337432	best: 0.6338658 (3)	total: 17.4s	remaining: 6m 58s
10:	learn: 0.6367925	test: 0.6345509	best: 0.6345509 (10)	total: 19.2s	remaining: 6m 57s
11:	learn: 0.6373336	test: 0.6351951	best: 0.635

93:	learn: 0.6665249	test: 0.6807379	best: 0.6807379 (93)	total: 3m 17s	remaining: 5m 27s
94:	learn: 0.6667344	test: 0.6810827	best: 0.6810827 (94)	total: 3m 20s	remaining: 5m 26s
95:	learn: 0.6670245	test: 0.6811664	best: 0.6811664 (95)	total: 3m 22s	remaining: 5m 24s
96:	learn: 0.6672073	test: 0.6810551	best: 0.6811664 (95)	total: 3m 24s	remaining: 5m 22s
97:	learn: 0.6675792	test: 0.6818056	best: 0.6818056 (97)	total: 3m 26s	remaining: 5m 20s
98:	learn: 0.6677589	test: 0.6818169	best: 0.6818169 (98)	total: 3m 28s	remaining: 5m 17s
99:	learn: 0.6678866	test: 0.6818115	best: 0.6818169 (98)	total: 3m 30s	remaining: 5m 15s
100:	learn: 0.6681231	test: 0.6821584	best: 0.6821584 (100)	total: 3m 32s	remaining: 5m 13s
101:	learn: 0.6681782	test: 0.6824125	best: 0.6824125 (101)	total: 3m 34s	remaining: 5m 11s
102:	learn: 0.6681958	test: 0.6821921	best: 0.6824125 (101)	total: 3m 36s	remaining: 5m 8s
103:	learn: 0.6683548	test: 0.6823089	best: 0.6824125 (101)	total: 3m 38s	remaining: 5m 6s
104:

183:	learn: 0.6790241	test: 0.6870841	best: 0.6871134 (179)	total: 6m 43s	remaining: 2m 24s
184:	learn: 0.6791602	test: 0.6871517	best: 0.6871517 (184)	total: 6m 45s	remaining: 2m 22s
185:	learn: 0.6792159	test: 0.6871059	best: 0.6871517 (184)	total: 6m 48s	remaining: 2m 20s
186:	learn: 0.6793634	test: 0.6871596	best: 0.6871596 (186)	total: 6m 50s	remaining: 2m 18s
187:	learn: 0.6796073	test: 0.6873544	best: 0.6873544 (187)	total: 6m 52s	remaining: 2m 15s
188:	learn: 0.6796610	test: 0.6873035	best: 0.6873544 (187)	total: 6m 54s	remaining: 2m 13s
189:	learn: 0.6797809	test: 0.6874144	best: 0.6874144 (189)	total: 6m 56s	remaining: 2m 11s
190:	learn: 0.6799592	test: 0.6875468	best: 0.6875468 (190)	total: 6m 58s	remaining: 2m 9s
191:	learn: 0.6801058	test: 0.6876434	best: 0.6876434 (191)	total: 7m	remaining: 2m 7s
192:	learn: 0.6801175	test: 0.6876463	best: 0.6876463 (192)	total: 7m 3s	remaining: 2m 4s
193:	learn: 0.6802314	test: 0.6876868	best: 0.6876868 (193)	total: 7m 5s	remaining: 2m 2

In [ ]:
cb_pred2 = sample_submission.copy()
cb_pred2['target'] = y_pred_final
cb_pred2.to_csv('catboost_final.csv', index = False)